# About the company

Bellabeat Company focuses on the creation of smart health products for its target audience, which are women. The products makes use of data collected such as sleep data, activity, stress and reproductive health to inform the users of their daily living, also to provide knowledge on their health lifestyle and suggest necessary improvements. The company was founded in 2013 and since then has thrived to provide improve the health of women through their smart devices.



# Business Task 

To analyse smart device usage in order to gain insights into how customers use non-Bellabeat smart devices. The Bellabeat Co-founder and Chief Creative Officer ,Urška Sršen, believes that the analyis of these data will give better knowledge and display areas where the company can experience growth.

# Guiding Questions for Analysis

* What are some trends in the smart device usage ?
* How could these trends apply to a Bellabeat customer ?
* How could these trends help influence Bellabeat Marketing strategy ?

# Data Preparation

* The data is obtained from the Public Domain. It is a dataset made available through Modius.
* It contains data ranging from Personal Tracker, heart rate, sleep monitoring etc of 30 users.
* The dataset is arranged in a wide format.

# Loading the Packages 

In [ ]:
library(tidyverse)
library(janitor)
library(skimr)
library(ggplot2)

# Importing the datasets

In [ ]:
dailyActivity_merged <- read.csv("../input/fitbit/Fitabase Data 4.12.16-5.12.16/dailyActivity_merged.csv")

dailyCalories_merged <- read_csv("../input/fitbit/Fitabase Data 4.12.16-5.12.16/dailyCalories_merged.csv")

dailyIntensities_merged <- read_csv("../input/fitbit/Fitabase Data 4.12.16-5.12.16/dailyIntensities_merged.csv")

dailySteps_merged <- read_csv("../input/fitbit/Fitabase Data 4.12.16-5.12.16/dailySteps_merged.csv")

sleepDay_merged <- read_csv("../input/fitbit/Fitabase Data 4.12.16-5.12.16/sleepDay_merged.csv")

heartrate_seconds_merged <- read_csv("../input/fitbit/Fitabase Data 4.12.16-5.12.16/heartrate_seconds_merged.csv")

hourlyIntensities_merged <- read_csv("../input/fitbit/Fitabase Data 4.12.16-5.12.16/hourlyIntensities_merged.csv")

In [ ]:
#To have a better view of the data, It was necessary to oraginize the in a single data frame using the merge function
merge_daily_1 <- merge(dailyActivity_merged, dailyCalories_merged, by = c("Id","Calories"))
merge_daily_2 <- merge(dailyIntensities_merged, dailyIntensities_merged, by = c("Id","ActivityDay","SedentaryMinutes", 
                                                                          "LightlyActiveMinutes","FairlyActiveMinutes",
                                                                          "VeryActiveMinutes", "SedentaryActiveDistance", 
                                                                          "LightActiveDistance", "ModeratelyActiveDistance", "VeryActiveDistance"))

merge_daily <- merge(merge_daily_1, merge_daily_2, by = c("Id", "ActivityDay", "SedentaryMinutes","LightlyActiveMinutes", 
                                                        "FairlyActiveMinutes", "VeryActiveMinutes", "SedentaryActiveDistance",
                                                        "LightActiveDistance", "ModeratelyActiveDistance", "VeryActiveDistance")) %>% 
  select(-ActivityDay) %>% rename(Date=ActivityDate)

daily_data <- merge(merge_daily, sleepDay_merged, by = "Id", all = TRUE) %>% 
  drop_na() %>% 
  select(-SleepDay, -TrackerDistance)

In [ ]:
#Let's briefly summarise our newly merged dataset
summary(daily_data)

In [ ]:
#For consistent naming
clean_names(daily_data)

In [ ]:
#minimum sedentary minutes
min(daily_data$SedentaryMinutes)

#maximum sedentary minutes
max(daily_data$SedentaryMinutes)

#average sedentary minutes in hours
mean(daily_data$SedentaryMinutes)/60

daily_data %>%  
  select(TotalSteps,
         TotalDistance,
         SedentaryMinutes) %>%
summary()

daily_data %>% 
  select(Calories) %>% 
  summary()


* From the above analysis, the minimmum sedentary time is 0( which is okay).
* The maximum sedentary time is 1440 minutes, which is approximately 6hours. The effect could be harmful if continously prolonged. It could result in cancer mortality, cardiovascular disease, type 2 diabetes, high blood pressure and other adverse effects.


# Visualisations

Let's do a bit of visualisations

In [ ]:
#average sedentary time by customer
avg_sed <- daily_data %>% 
  group_by(Id) %>% 
  summarise(mean_sed = mean(SedentaryMinutes))
#sum of sedentary time by each customer
sum_sed <- daily_data %>% 
  group_by(Id) %>% 
  summarise(total_sed = sum(SedentaryMinutes))
#Divide by 54000 because it consists of 30 customers for a whole month of 30 days
mean(sum_sed$total_sed)/54000

In [ ]:
ggplot(sum_sed, aes(x=Id, y=total_sed))+
  geom_point(color="blue")+geom_smooth(method="gam", formula = y ~s(x))+
  theme(axis.text.x = element_text(angle = 45))+labs(title = "Monthly Sedentary Minutes for each Customer")

* On average each users spends approximately spends 690 sendentary minutes in a month, which is approximately 11hrs per day
* *Further analysis by experst around the world shows that sitting time and activity levels found that those who sat for more than eight hours a day with no physical activity had a risk of dying similar to the risks of dying posed by obesity and smoking.*
* **Bellabeat company could create a system that will notify the users to take a walk or stand up once sedentary minutes is exceeding 8hrs.**

In [ ]:
ggplot(data=daily_data, aes(x=TotalMinutesAsleep, y=TotalTimeInBed)) + 
  geom_point(color="blue")+labs(title = "Variation of Total Minutes with Total Time in Bed")

* From the above analysis, there seems to be a somewhat linear reltionship between Total time in bed and the total minutes asleep. The total time spent in bed is almost equal to the total time asleep. **The company can create notifications or alarm messages for their devices to improve the sleep of its users making them aware of sleeping periods or rest.**

In [ ]:
#Separating the date and time using the separate function, so as to create separate columns for the Date and Time
new_hourly_intensity <- separate(hourlyIntensities_merged, ActivityHour, into =c("Date", "Time"), sep=" ")

In [ ]:
ggplot(new_hourly_intensity, aes(x=Time, y=TotalIntensity))+
  geom_col(color="red")+labs(title="Total Intensity against Time")+
  theme(axis.text.x = element_text(angle = 45))

* The above visualization illustrates that users are really active at hours between 5pm and 9pm, probably after the end of the day's work.
* **Bellabeat could use this opportunity to inform the users about taking a walk between the hours of 5pm and 7pm or exercising or jogging to improve the heart rate. ***
* Jogging reduces the risk at which heart attack occurs.

In [ ]:
ggplot(daily_data, aes(x= TotalSteps, y=Calories))+
  geom_point(color="red")+labs(title = "Number of Calories burned by Total Steps taken")

* The significance of this is quite clear the more steps we take, the more calories we burn. **Bellabeat could create a sense of motivation (through notifications or alerts) for users making them take more steps and reach their goals!**

# Recommendations and Conclusions

* **Recommendations for improvements**
1. Having understood the effect of sedentary time on the health of individuals, the company could focus on creating a system to notify its users to take a walk or jog when the sendentary time is exceeding 8hrs.
2. Creating an alert system could help inform the users to take a rest or sleep as their is a direct relationship between Total time in bed and Total time asleep.
3. Bellabeat could inform the users about taking a walk between the hours of 5pm and 7pm or exercising or jogging because analysis shows these are their active hours. This exercise could reduce the risk of having a heart attack.
4. Users could set goals on the number of calories they want to burn. Bellabeat will use this opportunity to motivate them by informing them on the amount of steps remaining for them to reach that goal.

![ cool ](https://img.freepik.com/free-photo/group-women-running-park_1301-7606.jpg?t=st=1649534686~exp=1649535286~hmac=cb7eb2c3c0d65659252bbfbe2d5c8993e495ac1dc991af122a5968cf61f6446a) ![]()

We these few recommendations the Bellabeat company will experience growth and better accuracy when making informed decisions and this will go a long way in showing the users that we value them and consider their health and well being as top most priority. The app is user friendly, easy to use and get perfect day by day. I also suggest a referral scheme, where users write their experiences with the product, for example how it helped them and they will get rewarded. This could boost help them develop a better marketing strategy and also boost their sales

How did i go ? Feel free to lay your comments, I will deeply appreciate this😊.

PS: My first Project